# Preparing the data

### The Process - Unstructured data to Vector Embeddings
![](./images/preparing_data.png)



## Setup OpenAI API

In [1]:
import os

import azure.identity
import dotenv
import openai

# Set up OpenAI client based on environment variables
dotenv.load_dotenv()
AZURE_OPENAI_SERVICE = os.getenv("AZURE_OPENAI_SERVICE")
AZURE_OPENAI_ADA_DEPLOYMENT = os.getenv("AZURE_OPENAI_ADA_DEPLOYMENT")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

azure_credential = azure.identity.AzureDeveloperCliCredential(
    tenant_id=os.getenv("AZURE_TENANT_ID"))
token_provider = azure.identity.get_bearer_token_provider(azure_credential,
                                                          "https://cognitiveservices.azure.com/.default")
openai_client = openai.AzureOpenAI(
    api_version="2024-06-01",
    azure_endpoint=f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com",

    azure_ad_token_provider=token_provider)

### Chunking and creating Embeddings

In [3]:
import json
import openai
import numpy as np
import faiss

# Load movie data
with open("/workspaces/rag-with-azure-ai-search-notebooks/movies.json", "r") as file:
    data = json.load(file)

# Step 1: Extract movie details
movies = data["movies"]

# Step 2: Create chunks of movie details
chunk_size = 1  # Number of movies per chunk
chunks = []

for i in range(0, len(movies), chunk_size):
    chunk = movies[i:i + chunk_size]
    chunk_text = "\n".join(
        f"Title: {movie['title']}, Plot: {movie['plot']}"
        for movie in chunk
    )
    chunks.append(chunk_text)

# Step 3: Display the chunks
print("Chunked Movie Details:")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i + 1}:\n{chunk}\n")

Chunked Movie Details:
Chunk 1:
Title: The Shawshank Redemption, Plot: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.

Chunk 2:
Title: Pulp Fiction, Plot: The lives of two mob hit men, a boxer, a gangster's wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Chunk 3:
Title: Forrest Gump, Plot: Forrest Gump, while not intelligent, has accidentally been present at many historic moments, but his true love, Jenny Curran, eludes him.

Chunk 4:
Title: The Lord of the Rings: The Return of the King, Plot: Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.

Chunk 5:
Title: The Lord of the Rings: The Fellowship of the Ring, Plot: A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle Earth from the Dark Lord Sauron.

Chunk 6:
Title

In [5]:
chunk_embeddings = []
for chunk in chunks:
    # Combine titles in the chunk into a single string
    chunk_text = " ".join(chunk)
    response = openai_client.embeddings.create(
        model=AZURE_OPENAI_ADA_DEPLOYMENT, input=chunk_text)
    embedding = response.data[0].embedding
    chunk_embeddings.append(embedding)

print("Chunk embeddings created", len(chunk_embeddings), "chunks.")
# Print first 5 dimensions of the first chunk embedding
print("First chunk embedding:", chunk_embeddings[0][:10])

Chunk embeddings created 25 chunks.
First chunk embedding: [-0.006355491001158953, -0.025520119816064835, 0.010039783082902431, -0.016083283349871635, 0.010187014937400818, 0.0034143675584346056, -0.031072847545146942, -0.02000945620238781, -0.007740167900919914, -0.003505510976538062]


### Why is length of all Embeddings = 1536?

In [6]:
len(chunk_embeddings[2])

1536

# Storing the Data

### Indexing

In [8]:

# Step 4: Index the embeddings using FAISS
dimension = len(chunk_embeddings[0])  # Dimensionality of the embeddings
index = faiss.IndexFlatL2(dimension)  # Create a flat (non-compressed) index

# Convert embeddings to a NumPy array and add to the index
embeddings_array = np.array(chunk_embeddings).astype('float32')
index.add(embeddings_array)

# Save the FAISS index for later use
faiss.write_index(index, "movie_title_embeddings.index")

print("Movie titles have been chunked, embedded, and indexed successfully!")

Movie titles have been chunked, embedded, and indexed successfully!


# Retrieval of Data

### Similarity Search

In [9]:
import faiss
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Load FAISS index
index = faiss.read_index("movie_title_embeddings.index")
dimension = index.d
num_vectors = index.ntotal

print(
    f"Loaded FAISS index with {num_vectors} vectors of dimension {dimension}")
assert len(
    movies) == num_vectors, "Number of titles must match number of embeddings in FAISS index."

# Retrieve all stored embeddings
embeddings = np.vstack([index.reconstruct(i) for i in range(num_vectors)])
print(f"Retrieved {embeddings.shape[0]} embeddings from index.")

# Define query
query_text = "soldier"
response = openai_client.embeddings.create(
    model=AZURE_OPENAI_ADA_DEPLOYMENT,
    input=[query_text]
)
query_vector = np.array(response.data[0].embedding).astype(
    'float32').reshape(1, -1)

# Compute cosine similarity
similarities = cosine_similarity(query_vector, embeddings)[0]

# Build DataFrame with titles and similarity scores
df = pd.DataFrame({
    "Title": movies,
    "Similarity": similarities
}).sort_values("Similarity", ascending=False)

print("\nTop 5 similar movies:")
print(df.head(5))

Loaded FAISS index with 25 vectors of dimension 1536
Retrieved 25 embeddings from index.

Top 5 similar movies:
                                                Title  Similarity
23  {'title': 'Iron Man', 'year': '2008', 'genres'...    0.770712
15  {'title': 'Schindler's List', 'year': '1993', ...    0.770570
9   {'title': 'The Matrix', 'year': '1999', 'genre...    0.770208
10  {'title': 'Gladiator', 'year': '2000', 'genres...    0.762998
18  {'title': 'Avatar', 'year': '2009', 'genres': ...    0.760081
